<a href="https://colab.research.google.com/github/LucasBezerraSantos/Alura_Machine_Learning/blob/master/Sistemas_de_recomendacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sistemas de Recomendação

Os sistemas de recomendação desempenham um papel importante em diversas áreas, especialmente no campo do comércio eletrônico, entretenimento, mídia social e serviços de streaming. Eles têm a finalidade de ajudar os usuários a descobrir novos produtos, conteúdos ou informações relevantes com base em suas preferências, histórico de interações e comportamento.

A importância dos sistemas de recomendação reside em vários aspectos:

Personalização: Os sistemas de recomendação oferecem aos usuários uma experiência personalizada, fornecendo recomendações sob medida com base em seus interesses individuais. Isso ajuda a melhorar a satisfação do usuário, tornando a interação mais relevante e significativa.

Descoberta de conteúdo: Com a enorme quantidade de informações disponíveis atualmente, os sistemas de recomendação ajudam os usuários a encontrar conteúdos relevantes que de outra forma poderiam passar despercebidos. Isso permite que os usuários descubram novos produtos, filmes, músicas, livros e outros tipos de conteúdo que sejam do seu interesse.

Aumento do engajamento e fidelidade do usuário: Ao fornecer recomendações relevantes, os sistemas de recomendação aumentam o engajamento do usuário, incentivando-o a explorar mais conteúdos e serviços oferecidos pela plataforma. Isso também contribui para a fidelização do usuário, pois ele percebe que suas preferências são levadas em consideração.

Aumento das vendas e receitas: Em ambientes de comércio eletrônico, os sistemas de recomendação podem impulsionar as vendas ao direcionar os usuários para produtos relacionados ou complementares aos que eles já estão interessados. Isso pode resultar em um aumento das receitas e uma melhor experiência de compra para os usuários.

Suporte à tomada de decisão: Além de recomendações de produtos ou conteúdos, os sistemas de recomendação também podem ser aplicados em outros domínios, como recomendações de rotas de transporte, sugestões de amigos em redes sociais ou recomendações de notícias relevantes. Isso auxilia os usuários na tomada de decisões informadas e eficientes.

Em resumo, os sistemas de recomendação desempenham um papel crucial na era da informação, ajudando os usuários a lidar com a sobrecarga de informações e a descobrir novas opções relevantes. Eles aprimoram a experiência do usuário, aumentam o engajamento, impulsionam as vendas e fornecem benefícios tanto para os usuários quanto para as empresas que os implementam.

### Leitura de Dados

In [1]:
import pandas as pd
import numpy as np

In [2]:
filmes = pd.read_csv('/content/sample_data/movies.csv')
filmes.columns = ['filmeID','titulo','genero']
filmes = filmes.set_index('filmeID')
filmes.head()

,titulo,genero
filmeID,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [3]:
notas = pd.read_csv('/content/sample_data/ratings.csv')
notas.columns = ['usuarioID','filmeID','pontuacao', 'momento']
notas.head()

,usuarioID,filmeID,pontuacao,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## Primeira Recomendação: Popularidade

In [4]:
total_avaliacoes = notas.filmeID.value_counts()
filmes['avaliacoes'] = total_avaliacoes

In [5]:
filmes.head()

,titulo,genero,avaliacoes
filmeID,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
2,Jumanji (1995),Adventure|Children|Fantasy,110.0
3,Grumpier Old Men (1995),Comedy|Romance,52.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
5,Father of the Bride Part II (1995),Comedy,49.0


**Filmes mais avaliados**

In [6]:
filmes.sort_values('avaliacoes', ascending=False).head()

,titulo,genero,avaliacoes
filmeID,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0


**Pontuação média dos usuários**

In [7]:
pontuacao_media = notas.groupby('filmeID').mean()['pontuacao']
filmes['pontuacao_media'] = pontuacao_media
filmes.sort_values('pontuacao_media', ascending=False).head(10)

,titulo,genero,avaliacoes,pontuacao_media
filmeID,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0
6201,Lady Jane (1986),Drama|Romance,1.0,5.0
102217,Bill Hicks: Revelations (1993),Comedy,1.0,5.0
102084,Justice League: Doom (2012),Action|Animation|Fantasy,1.0,5.0
6192,Open Hearts (Elsker dig for evigt) (2002),Romance,1.0,5.0


## Segunda Recomendação: Nota média

In [8]:
filmes_com_mais_de_60_votos = filmes.query('avaliacoes > 60')
filmes_com_mais_de_60_votos.sort_values('pontuacao_media', ascending=False).head(10)

,titulo,genero,avaliacoes,pontuacao_media
filmeID,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000
912,Casablanca (1942),Drama|Romance,100.0,4.240000


## Terceira Recomendação: Gênero

In [19]:
genero_busca = 'Adventure|Children|Fantasy'

In [20]:
filmes_com_mais_de_60_votos.query("genero == @genero_busca").sort_values('pontuacao_media', ascending=False).head(5)

,titulo,genero,avaliacoes,pontuacao_media
filmeID,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,107.0,3.761682
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,62.0,3.443548
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818


## Quarta Recomendação: Similaridade

**Avaliando notas do usuário**

A norma de um vetor ou matriz fornece uma medida da **distância** ou tamanho do objeto no espaço.

A função np.linalg.norm do NumPy é usada para calcular a norma de um vetor ou matriz. A norma é uma medida de magnitude ou comprimento de um vetor ou matriz. Existem diferentes tipos de normas que podem ser calculadas, como a norma Euclidiana, norma Manhattan, norma máxima, entre outras. O tipo de norma é especificado como parâmetro da função. Em resumo, a função np.linalg.norm permite calcular a norma de um vetor ou matriz de forma fácil e conveniente utilizando o NumPy.

In [11]:
def distancia_de_vetores(a,b):
  return np.linalg.norm(a - b)

Seleção de notas para um usuário específico

In [12]:
def notas_do_usuario(usuario):
  notas_do_usuario = notas.query('usuarioID== %d' %usuario)
  notas_do_usuario = notas_do_usuario[['filmeID', 'pontuacao']].set_index('filmeID')
  return notas_do_usuario

Função para criação de vetores comparativos entre as notas de um usuário específico e os demais usuários.

In [13]:
def pontuacao_usuarios(usuario_id1, usuario_id2, minimo = 5):
  notas1 = notas_do_usuario(usuario_id1)
  notas2 = notas_do_usuario(usuario_id2)
  comparativo = notas1.join(notas2, lsuffix='_esquerda', rsuffix='_direita').dropna()

  if (len(comparativo) < minimo):
    return [usuario_id1, usuario_id2, 50000]

  distancia = distancia_de_vetores(comparativo['pontuacao_direita'], comparativo['pontuacao_esquerda'])
  return [usuario_id1, usuario_id2, round(distancia, 4)]

Função para cálculo de distância entre um usuário e os demais usuários do conjunto de dados

In [14]:
def distancia_entre_todos(usuario_avaliado, k_mais_proximos=10):
  todos_os_usuarios = notas.usuarioID.unique()
  distancias = [pontuacao_usuarios(usuario_avaliado, usuario_id) for usuario_id in todos_os_usuarios]
  distancias = pd.DataFrame(distancias, columns = ['usuario_avaliado', 'outros_usuarios', 'distancia'])
  
  distancias = distancias.set_index('outros_usuarios').drop(usuario_avaliado)
  distancias = distancias.sort_values('distancia')
  return distancias.head(k_mais_proximos)

Função para identificar usuários próximos, a nota média que atribuíram para os filmes, considerando um número de avaliações inicial para recomendação e ordenando pela maior pontuação.

In [15]:
def sugere_para(voce, k_mais_proximos = 10, n_recomendacoes=None):
  notas_de_voce = notas_do_usuario(voce)
  filmes_que_voce_ja_viu = notas_de_voce.index

  similares = distancia_entre_todos(voce, k_mais_proximos = k_mais_proximos)
  usuarios_similares = similares.index
  notas_dos_similares = notas.set_index("usuarioID").loc[usuarios_similares]
  notas_dos_similares = notas_dos_similares.rename(columns={'pontuacao': 'pontuacao_usuarios_proximos'})
  
  recomendacoes = notas_dos_similares.groupby("filmeID").mean()[["pontuacao_usuarios_proximos"]]
  recomendacoes['aparicoes_usuarios_proximos'] = notas_dos_similares.groupby("filmeID").count()[['pontuacao_usuarios_proximos']]
  filtro_avaliacoes = k_mais_proximos/2

  recomendacoes = recomendacoes.join(filmes)
  recomendacoes = recomendacoes.query("aparicoes_usuarios_proximos >= %.2f" % filtro_avaliacoes)  

  recomendacoes = recomendacoes.sort_values(by=["pontuacao_usuarios_proximos", 'avaliacoes'], ascending=False)
  recomendacoes.query("avaliacoes >= 10", inplace=True)

  return recomendacoes.head(n_recomendacoes)

In [16]:
sugere_para(5, 20, 5)

,pontuacao_usuarios_proximos,aparicoes_usuarios_proximos,titulo,genero,avaliacoes,pontuacao_media
filmeID,,,,,,
296,4.653846,13,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
527,4.454545,11,Schindler's List (1993),Drama|War,220.0,4.225000
356,4.416667,12,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
2959,4.333333,12,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
318,4.227273,11,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
